In [45]:
### Conda env for this is REST. Use $ source activate REST
import sys
if sys.version_info[0] < 3:
    from StringIO import StringIO
else:
    from io import StringIO
import requests
import setup as mongler
import pandas as pd
import time as time
from pymongo import MongoClient

## Configure mongo
mongo_client = MongoClient(mongler.MONGO_ADDRESS,mongler.MONGO_PORT)
reportsDB = mongo_client.BYUbridge.reports

## Make requests to bridge for list of reports
headers = {"Authorization":"Token %s" % mongler.TOKEN}

## returns all of the reports on the bridge that contain an exact match to a keyword
def GetReportList(keyword):
    reports = []
    pageNum = 1
    while True:
        payload = {'page':pageNum, 'perPage':'100'}
        r = requests.get('https://byu-csm.symplicity.com/api/public/v1/reports', params = payload, headers=headers)
        models = r.json()['models']
        if len(models) == 0:
            break
        else:
            reports.append(models)
            pageNum += 1

    ### Find reports with the name ADRIEL in the Label
    keywordReports = []
    for page in reports:
        for report in page:
            if keyword in report['label']:
                keywordReports.append(report)

    ## This will print all of the reports
    return keywordReports

## Run the desired report
def RunReport(reportID, headers, payload):
    request = requests.put('https://byu-csm.symplicity.com/api/public/v1/reports/%s/run' %reportID, headers=headers)
    ## This while loop waits until the most recent report to be completed
    while True:
        time.sleep(6)
        ## this payload will request only the most recent run
        tmp_payload = {'page':'1', 'perPage':'1'}
        r = requests.get('https://byu-csm.symplicity.com/api/public/v1/reports/%s/runs' %reportID, params = tmp_payload, headers=headers)
        tmp = r.json()
        if tmp['models'][0]['label'] == 'complete':
            break
    ## Once the report is run and completed, get the report run data
    request = requests.get('https://byu-csm.symplicity.com/api/public/v1/reports/%s/data' %reportID, headers=headers, params=payload)
    TMPDATA=StringIO(request.text)
    return pd.read_csv(TMPDATA)

## This Main function will run all the desired reports given a certain keyword
def main():
    ## Get the list of Reports
    adrielReports = GetReportList('ADRIEL')
    print(adrielReports)

    ## Name all of the reports based on label
    for i, report in enumerate(adrielReports):
        if 'Full Student List' in report['label']:
            fullStudent = report
        elif 'Archived Events' in report['label']:
            archivedAttendees = report
        elif 'Non-archived' in report['label']:
            nonArchivedEvents = report
        else:
            print("Named %n reports" %(i +1))
            break

    ## Run reports
    payload = {'format':'csv'}
    studentReport = RunReport(fullStudent['id'], headers, payload)
    attendeeReport = RunReport(archivedAttendees['id'], headers, payload)
    attendeeReport2 = RunReport(nonArchivedEvents['id'], headers, payload)

    ## Append the attendee reports and merge
    attendeeReport = attendeeReport.append(attendeeReport2)
    attendeeReport = attendeeReport.merge(studentReport, left_on='Kiosk Swipe Log: student', right_on='Name')
    attendeeReport.to_csv(path_or_buf = '~/MAIN/BCC/Club\ data/attendeeReport.csv')

In [46]:
adrielReports = GetReportList('ADRIEL')

In [47]:
for i, report in enumerate(adrielReports):
    if 'Full Student List' in report['label']:
            fullStudent = report
    elif 'Archived Events' in report['label']:
            archivedAttendees = report
    elif 'Non-archived' in report['label']:
            nonArchivedEvents = report
    else:
        print("Named %n reports, ERROR" %(i +1))
        break

In [48]:
payload = {'format':'csv'}

In [54]:
attendeeReport = RunReport(nonArchivedEvents['id'], headers, payload)
attendeeReport2 = RunReport(archivedAttendees['id'], headers, payload)

In [56]:
attendeeReport2

,Information Session: Information Session Type,Information Session: Start Date/Time,Information Session: Employer,Information Session: Club Event Title,Kiosk Swipe Log: student
0,Marriott School Club Events,"Sep 01, 2017, 4:00 PM",Women in Business Club (WIB),WIB New Student Orientation Reception,Abraham Nephi Bedard
1,Marriott School Club Events,"Sep 01, 2017, 4:00 PM",Women in Business Club (WIB),WIB New Student Orientation Reception,Andrew Joseph Campbell
2,Marriott School Club Events,"Sep 01, 2017, 4:00 PM",Women in Business Club (WIB),WIB New Student Orientation Reception,Anna Elise Caldwell
3,Marriott School Club Events,"Sep 01, 2017, 4:00 PM",Women in Business Club (WIB),WIB New Student Orientation Reception,Braden Trent Erickson
4,Marriott School Club Events,"Sep 01, 2017, 4:00 PM",Women in Business Club (WIB),WIB New Student Orientation Reception,Brandon Jay Christensen
5,Marriott School Club Events,"Sep 01, 2017, 4:00 PM",Women in Business Club (WIB),WIB New Student Orientation Reception,Brandon Morgan Milne
6,Marriott School Club Events,"Sep 01, 2017, 4:00 PM",Women in Business Club (WIB),WIB New Student Orientation Reception,Caden John Wilding
7,Marriott School Club Events,"Sep 01, 2017, 4:00 PM",Women in Business Club (WIB),WIB New Student Orientation Reception,Cameron Matthew Powers
8,Marriott School Club Events,"Sep 01, 2017, 4:00 PM",Women in Business Club (WIB),WIB New Student Orientation Reception,Cedric William Huntington
9,Marriott School Club Events,"Sep 01, 2017, 4:00 PM",Women in Business Club (WIB),WIB New Student Orientation Reception,Chloe Beth Haderlie


In [57]:
attendeeReport3 = attendeeReport.append(attendeeReport2)

In [58]:
attendeeReport3

,Information Session: Information Session Type,Information Session: Start Date/Time,Information Session: Employer,Information Session: Club Event Title,Kiosk Swipe Log: student
0,Marriott School Club Events,"Sep 26, 2017, 3:00 PM",Experience Design and Management Club (ExDM),ExDM Junior Core Student Panel,Colby Shawn Nelson
1,Marriott School Club Events,"Sep 26, 2017, 3:00 PM",Experience Design and Management Club (ExDM),ExDM Junior Core Student Panel,Dallas Gregg Anderson
2,Marriott School Club Events,"Sep 26, 2017, 3:00 PM",Experience Design and Management Club (ExDM),ExDM Junior Core Student Panel,Emily Anderson
3,Marriott School Club Events,"Sep 26, 2017, 3:00 PM",Experience Design and Management Club (ExDM),ExDM Junior Core Student Panel,Emily Suzanne Crowder
4,Marriott School Club Events,"Sep 26, 2017, 3:00 PM",Experience Design and Management Club (ExDM),ExDM Junior Core Student Panel,Heather Sue Houghton
5,Marriott School Club Events,"Sep 26, 2017, 3:00 PM",Experience Design and Management Club (ExDM),ExDM Junior Core Student Panel,Julia Nicole Stolt
6,Marriott School Club Events,"Sep 26, 2017, 3:00 PM",Experience Design and Management Club (ExDM),ExDM Junior Core Student Panel,Kristopher Isaac King
7,Marriott School Club Events,"Sep 26, 2017, 3:00 PM",Experience Design and Management Club (ExDM),ExDM Junior Core Student Panel,Laurel Annette Johnson
8,Marriott School Club Events,"Sep 26, 2017, 3:00 PM",Experience Design and Management Club (ExDM),ExDM Junior Core Student Panel,Lisa Stringham
9,Marriott School Club Events,"Sep 26, 2017, 3:00 PM",Experience Design and Management Club (ExDM),ExDM Junior Core Student Panel,Madison Marie Fullmer Hunt


In [43]:
df = pd.read_csv(TESTDATA, header = 0)

In [44]:
df

,Name,Email,First Name,Last Name,Graduation Date,Student ID
0,Christopher Matthew Melville,cmelville23@gmail.com,Christopher Matthew,Melville,"Dec 01, 2017",398678447
1,Jacob Leonard Marshall,newzealandcougar@gmail.com,Jacob,Marshall,"Dec 01, 2017",107664349
2,Ivan Edward Bowles,ivanebowles@gmail.com,Ivan Edward,Bowles,"Dec 01, 2017",368869312
3,Jenny Logan Lapides,jlapides@ymail.com,Jenny Logan,Lapides,"Dec 01, 2017",152918578
4,Alex W La,la.alexw@gmail.com,Alex,La,"Dec 01, 2017",084204896
5,Patrick Spandre Badger,patrickbadger1991@hotmail.com,Patrick Spandre,Badger,"Dec 01, 2017",692929829
6,Sarah Christine Berry,sarahberry92@gmail.com,Sarah Christine,Berry,"Dec 01, 2017",075364693
7,William Thomas Keele,william.keele103@gmail.com,William Thomas,Keele,"Dec 01, 2017",620080240
8,Jordyn Alan Dickey,dickeyjordyn@yahoo.com,Jordyn Alan,Dickey,"Dec 01, 2017",023639658
9,Jordan Marie Crowley,jordynmarie@gmail.com,Jordan,Crowley,"Dec 01, 2017",463722287


In [ ]:
request.json()